In [ ]:
import scirpy as ir

In [ ]:
ir.__version__

In [ ]:
import scanpy as sc

In [ ]:
from glob import glob
import warnings
import tarfile

import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc

In [ ]:
import muon as mu

In [ ]:
import numpy as np

In [ ]:
samples = {
    "10mix1": {"group": "10mix"},
    "10mix2": {"group": "10mix"},
    "11mix1": {"group": "11mix"},
    "11mix2": {"group": "11mix"},
    "GF1": {"group": "GF"},
    "GF2": {"group": "GF"},

}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr = {}
adatas_gex = {}
for sample, sample_meta in samples.items():
    #adata_gex = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    #adata_gex.var_names_make_unique()
    adatas_tcr[sample] = adata_tcr
    #adatas_gex[sample] = adata_gex

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},


}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr_2021 = {}
#adatas_gex_2021 = {}
for sample, sample_meta in samples.items():
    #adata_gex_2021 = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr_2021 = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    #adata_gex_2021.var_names_make_unique()
    adatas_tcr_2021[sample] = adata_tcr_2021
    #adatas_gex_2021[sample] = adata_gex_2021

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},
    "10mix1": {"group": "10mix", "condition":"naive"},
    "10mix2": {"group": "10mix", "condition":"naive"},
    "11mix1": {"group": "11mix", "condition":"naive"},
    "11mix2": {"group": "11mix", "condition":"naive"},
    "GF1": {"group": "GF", "condition":"naive"},
    "GF2": {"group": "GF", "condition":"naive"},
}    

### TCR 2019 NORMAL

In [ ]:
adata_tcr

In [ ]:
# Merge anndata objects
adata_tcr = anndata.concat(adatas_tcr, index_unique="_")

In [ ]:
adata_tcr.obs

### TCR 2021 TUMOR

In [ ]:
adata_tcr_2021

In [ ]:
# Merge anndata objects
adata_tcr_2021 = anndata.concat(adatas_tcr_2021, index_unique="_")

In [ ]:
adata_tcr_2021.obs

### Load GEX

In [ ]:
adata = sc.read_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/13_january_2025/mapped_data/mapped_til_colon.h5ad")

In [ ]:
adata.var

In [ ]:
adata.obs.origin =  adata.obs.origin.replace("colon","normal")

In [ ]:
adata.obs.origin =  adata.obs.origin.replace("til","tumor")

In [ ]:
adata_normal = adata[adata.obs["origin"]=="normal"]

In [ ]:
adata_tumor = adata[adata.obs["origin"]=="tumor"]

### Transform TCR 2019 NORMAL

In [ ]:
adata_tcr.obs["sample_id"] = adata_tcr.obs_names.to_series().str.split("_", expand=True)[1]

In [ ]:
adata_tcr.obs["group"] = adata_tcr.obs["sample_id"].map(lambda x: samples[x]["group"])
adata_tcr.obs["condition"] = adata_tcr.obs["sample_id"].map(lambda x: samples[x]["condition"])

In [ ]:
adata_tcr.obs

### Transform TCR 2021 TUMOR

In [ ]:
adata_tcr_2021.obs["sample_id"] = adata_tcr_2021.obs_names.to_series().str.split("_", expand=True)[1]

In [ ]:
adata_tcr_2021.obs["group"] = adata_tcr_2021.obs["sample_id"].map(lambda x: samples[x]["group"])
adata_tcr_2021.obs["condition"] = adata_tcr_2021.obs["sample_id"].map(lambda x: samples[x]["condition"])

In [ ]:
adata_tcr_2021.obs

### Merge TCR TUMOR & NORMAL

In [ ]:
adata_tcr_merge = anndata.concat([adata_tcr,adata_tcr_2021], index_unique="_")

In [ ]:
adata_tcr_merge.obs

### Merge GEX TUMOR & NORMAL

In [ ]:
adata_gex_merge = anndata.concat([adata_normal,adata_tumor], index_unique="_")

In [ ]:
adata_gex_merge.obs

In [ ]:
adata_gex_merge

In [ ]:
# Check if 'Ifng' is in the variable names
if "ENSMUSG00000055170" in adata_gex_merge.var_names:
    # Safely extract the expression values for 'Ifng'
    ifng_expression = (
        adata_gex_merge[:, "ENSMUSG00000055170"].X.toarray().flatten()
        if hasattr(adata_gex_merge[:, "ENSMUSG00000055170"].X, "toarray")
        else adata[:, "ENSMUSG00000055170"].X.flatten()
    )

    # Add Ifng expression status to adata.obs
    adata_gex_merge.obs["Ifng_expression"] = (ifng_expression > 6).astype(int)

    # Create a new column 'ifng' based on 'Ifng_expression'
    adata_gex_merge.obs["ifng"] = adata_gex_merge.obs["Ifng_expression"].apply(
        lambda x: "pos" if x == 1 else "neg"
    )
else:
    print("Gene 'Ifng' not found in adata.var_names")

In [ ]:
adata_tcr_merge

In [ ]:
# Check if 'Ifng' is in the variable names
if "ENSMUSG00000055170" in adata_tcr_merge.var_names:
    # Safely extract the expression values for 'Ifng'
    ifng_expression = (
        adata_tcr_merge[:, "ENSMUSG00000055170"].X.toarray().flatten()
        if hasattr(adata_tcr_merge[:, "ENSMUSG00000055170"].X, "toarray")
        else adata[:, "ENSMUSG00000055170"].X.flatten()
    )

    # Add Ifng expression status to adata.obs
    adata_tcr_merge.obs["Ifng_expression"] = (ifng_expression > 6).astype(int)

    # Create a new column 'ifng' based on 'Ifng_expression'
    adata_tcr_merge.obs["ifng"] = adata_tcr_merge.obs["Ifng_expression"].apply(
        lambda x: "pos" if x == 1 else "neg"
    )
else:
    print("Gene 'Ifng' not found in adata.var_names")

### Create MUDATA

In [ ]:
mdata = mu.MuData({"gex": adata_gex_merge, "airr": adata_tcr_merge})

### Transfor obs names for gex and tcr matching

In [ ]:
mdata.obs.index = mdata.obs.index.str.replace('-', '_')

In [ ]:
# Replace "-" with "_" in the 'obs_names' column
mdata["airr"].obs_names = mdata["airr"].obs_names.str.replace('-', '_')

In [ ]:
# Replace "-" with "_" in the 'obs_names' column
mdata["gex"].obs_names = mdata["gex"].obs_names.str.replace('-', '_')

In [ ]:
len(set(mdata["gex"].obs_names) & set(mdata["airr"].obs_names))

In [ ]:
mdata

In [ ]:
sc.pl.umap(mdata["gex"], color = "leiden")

### Create MDATA GEX & TCR 2019 NORMAL

In [ ]:
mdata_normal = mu.MuData({"gex": adata_normal, "airr": adata_tcr})

In [ ]:
mdata_normal

In [ ]:
sc.pl.umap(mdata_normal["gex"], color = "leiden")

### Create MDATA GEX & TCR 2021 TUMOR

In [ ]:
mdata_tumor= mu.MuData({"gex": adata_tumor, "airr": adata_tcr_2021})

In [ ]:
mdata_tumor

In [ ]:
sc.pl.umap(mdata_tumor["gex"], color = "leiden")

### Transform mdata

In [ ]:
sc.pp.filter_genes(mdata["gex"], min_cells=500)
sc.pp.filter_cells(mdata["gex"], min_genes=500)

In [ ]:
sc.pp.log1p(mdata["gex"])
sc.pp.pca(mdata["gex"], svd_solver="arpack")
sc.pp.neighbors(mdata["gex"])
sc.tl.umap(mdata["gex"])

In [ ]:
ir.pp.index_chains(mdata)
ir.tl.chain_qc(mdata)

In [ ]:
# Define the conditions
normal_samples = ["10mix1", "10mix2", "11mix1", "11mix2", "GF1", "GF2"]

# Use np.where() to create the 'origin' column based on conditions
mdata["airr"].obs["origin"] = mdata["airr"].obs["sample_id"].apply(
    lambda x: "colon" if x in normal_samples else "tumor"
)

In [ ]:
mdata

In [ ]:
mdata["gex"].obs.index

In [ ]:
mdata["airr"].obs.index

In [ ]:
mdata.obs.index = mdata.obs.index.str.replace('-', '_', regex=False)


In [ ]:
mdata["airr"].obs

In [ ]:
mdata.obs

In [ ]:
mdata.update()

In [ ]:
#mdata = mu.MuData({"gex": adata_gex_merge, "airr": adata_tcr_merge})

In [ ]:
mdata["gex"].var

In [ ]:
mdata["gex"].obs.cell_type = mdata["gex"].obs.cell_type.replace("nan","CD8_NaiveLike")

In [ ]:
mdata.update()

In [ ]:
mdata = mu.MuData({"gex": adata_gex_merge, "airr": adata_tcr_merge})

In [ ]:
fig, (ax0, ax1,ax2,ax3) = plt.subplots(1, 4, figsize=(20, 4), gridspec_kw={"wspace": 0.5})
mu.pl.embedding(mdata, basis="gex:umap", color=["ENSMUSG00000055170"], ax=ax0, show=False, frameon=False, cmap= "Reds")
mu.pl.embedding(mdata, basis="gex:umap", color=["airr:origin"], ax=ax1, show=False, frameon=False)
mu.pl.embedding(mdata, basis="gex:umap", color=["gex:cell_type"], ax=ax2, show=False, frameon=False)
mu.pl.embedding(mdata, basis="gex:umap", color=["airr:receptor_subtype"], ax=ax3, frameon=False)

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2,figsize=(10, 5) ,gridspec_kw={"wspace": 0.5})
mu.pl.embedding(mdata, basis="gex:umap", color=["ENSMUSG00000055170"], ax=ax0, show=False, frameon=False, cmap= "Reds")
mu.pl.embedding(mdata, basis="gex:umap", color=["gex:cell_type"], ax=ax1, show=False, frameon=False)


In [ ]:
mdata["gex"].obs

In [ ]:
mdata

In [ ]:
mdata["airr"].obs

## Daniel borras


In [ ]:
mdata["gex"].obs.ifng.value_counts()

In [ ]:
mdata["airr"].obs["ifng"] = mdata["gex"].obs["ifng"]

In [ ]:
ir.tl.alpha_diversity(mdata,  groupby="group", target_col="sample_id", metric="normalized_shannon_entropy")

In [ ]:
#tcr_og = mdata["airr"]

In [ ]:
tcr = mdata["airr"][mdata["airr"].obs["ifng"] == "pos"]


In [ ]:
tcr_10mix = tcr[tcr.obs["sample_id"].isin(["10mix1","10mix2","10mix-ICI1","10mix-ICI2"])]

In [ ]:
tcr_11mix = tcr[tcr.obs["sample_id"].isin(["11mix1","11mix2","11mix-ICI1","11mix-ICI2"])]

In [ ]:
tcr.obs.sample_id.value_counts()

In [ ]:
#tcr_11mix = tcr[tcr.obs["group"].isin(["11mix"])]

In [ ]:
tcr.obs.ifng.value_counts()

In [ ]:
tcr_10mix.obs["cell_count"]=1
tcr_11mix.obs["cell_count"]=1

In [ ]:
test=sc.concat([tcr_10mix,tcr_11mix])

In [ ]:
test.obs[["group","ifng","receptor_subtype","chain_pairing"]].to_numpy()

In [ ]:
test.obs["group"].to_numpy()

In [ ]:
pd.crosstab(columns=[test.obs["chain_pairing"], test.obs["ifng"]], index=test.obs["group"])

In [ ]:
ir.tl.define_clonotypes(test, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.pp.ir_dist(
    test,
    metric="alignment",
    sequence="aa",
    cutoff=15,
)

In [ ]:
ir.tl.define_clonotype_clusters(test, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any")

In [ ]:
ir.tl.clonotype_network(test, min_cells=2, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(test, color="sample_id", label_fontsize=5, panel_size=(6, 5), base_size=4,
                       title="tcr_10mix clonotypes")

In [ ]:
tcr_copy = test.copy()

In [ ]:
tcr_copy.obs

In [ ]:
# Count occurrences of each clono_id
clonotype_counts = test.obs["clone_id"].value_counts()

# Map the counts back to a new column in obs
test.obs["count_clonotype_id"] = test.obs["clone_id"].map(clonotype_counts)

# Check the result
print(test.obs[["clone_id", "count_clonotype_id"]].head())

In [ ]:
#tcr_10mix.obs.groupby("clone_id").filter(lambda x: x["sample_id"].nunique() >= 2)

In [ ]:
if "count_clonotype_id" not in test.obs.columns:
    test.obs["count_clonotype_id"] = test.obs["clone_id"].map(tcr.obs["clone_id"].value_counts())

# Filter the AnnData object
test_filtered = test[test.obs["count_clonotype_id"] > 2].copy()


In [ ]:
ir.pl.clonotype_network(test_filtered, color="chain_pairing", label_fontsize=5, panel_size=(6, 5), base_size=3,
                       title="tcr_10mix clonotypes")

In [ ]:
ir.tl.define_clonotypes(test_filtered, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.pp.ir_dist(
    test_filtered,
    metric="alignment",
    sequence="aa",
    cutoff=15,
)

In [ ]:
ir.tl.define_clonotype_clusters(test_filtered, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any")

In [ ]:
ir.tl.clonotype_network(test_filtered, min_cells=2, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(test_filtered, color="chain_pairing", label_fontsize=5, panel_size=(6, 5), base_size=1.8,
                       title="Clonotypes")

In [ ]:
ir.pl.clonotype_network(test_filtered, color="sample_id", label_fontsize=5, panel_size=(6, 5), base_size=5.8,size_power=1.4,
                       title="Clonotypes")

In [ ]:
tcr_10mix = test_filtered[test_filtered.obs["sample_id"].isin(["10mix1","10mix2","10mix-ICI1","10mix-ICI2"])]

In [ ]:
tcr_11mix = test_filtered[test_filtered.obs["sample_id"].isin(["11mix1","11mix2","11mix-ICI1","11mix-ICI2"])]

In [ ]:
tcr_10mix.obs.sample_id

In [ ]:
test_filtered.obs.sample_id

In [ ]:
tcr_11mix.obs.clone_id_size.mean()

In [ ]:
tcr_10mix.obs.clone_id_size.mean()

In [ ]:
test=pd.concat([tcr_10mix.obs,tcr_11mix.obs])

In [ ]:
tcr_copy

In [ ]:
g=sns.barplot(data=tcr_copy.obs, y="clone_id_size", hue="group", x="origin", orient="v", palette=["tab:orange","tab:blue"], width=0.8)

In [ ]:
test_filtered_normal

In [ ]:
test

In [ ]:
# Ensure indices match between `test` and `mdata["gex"].obs`
shared_index = test.index.intersection(mdata["gex"].obs.index)

# Extract "cell_type" and "origin" from `mdata["gex"].obs`, renaming to avoid conflicts
metadata_subset = mdata["gex"].obs.loc[shared_index, ["cell_type", "origin"]].copy()
metadata_subset = metadata_subset.rename(columns={"cell_type": "cell_type_new", "origin": "origin_new"})

# Merge the new columns into `test`
test_filtered.obs = test_filtered.obs.join(metadata_subset)

# Print confirmation
print(f"Updated test shape: {test.shape}")
print(test.head())  # Preview the updated dataframe


In [ ]:
test_filtered.obs

In [ ]:
g=sns.catplot(data=test[test.clone_id_dedup], y="clone_id_size", x="group",  col="chain_pairing",orient="v", sharey=False,
            height=2, aspect=1, kind="violin", palette=["tab:orange","tab:blue"], margin_titles=True)


In [ ]:
tcr_10mix.obs.chain_pairing

In [ ]:
test_filtered.obs

In [ ]:
test_filtered_tumor = test_filtered[test_filtered.obs["origin"].isin(["tumor"])]

In [ ]:
test_filtered_tumor

In [ ]:
test_filtered_normal = test_filtered[test_filtered.obs["origin"].isin(["colon"])]

In [ ]:
test_filtered_normal

In [ ]:
sc.pl.dotplot(test_filtered, var_names = ["clone_id_size"],groupby="cell_type_new", cmap="viridis")

In [ ]:
sc.pl.dotplot(test_filtered_normal, var_names = ["clone_id_size"],groupby="group")

In [ ]:
metricsPersample=pd.DataFrame(columns=["group","Richness"])
metricsPersample["Richness"]=list(ir.tl.alpha_diversity(tcr_11mix, groupby="sample_id", target_col="clone_id", metric="normalized_shannon_entropy", inplace=False)[0])

In [ ]:
persample11mix=list(ir.tl.alpha_diversity(tcr_11mix, groupby="sample_id", target_col="clone_id", metric="normalized_shannon_entropy", inplace=False)[0])

In [ ]:
persample11mix

In [ ]:
tcr_11mix.obs.sample_id.value_counts()

In [ ]:
metricsPersample["x"]="CRC"

In [ ]:
metricsPersample["group"]= list(tcr_11mix.obs.sample_id.unique())

In [ ]:
metricsPersample

In [ ]:
metricsPersample2=pd.DataFrame(columns=["group","Richness"])
metricsPersample2["Richness"]=list(ir.tl.alpha_diversity(tcr_10mix, groupby="sample_id", target_col="clone_id", metric="normalized_shannon_entropy", inplace=False)[0])

In [ ]:
persample10mix=list(ir.tl.alpha_diversity(tcr_10mix, groupby="sample_id", target_col="clone_id", metric="normalized_shannon_entropy", inplace=False)[0])

In [ ]:
metricsPersample2["group"]= list(tcr_10mix.obs.sample_id.unique())

In [ ]:
metricsPersample2["x"]="CRC"

In [ ]:
metricsPersample2

In [ ]:
metricsPS = pd.concat([metricsPersample, metricsPersample2], ignore_index=True, axis=0, join="outer")

In [ ]:
metricsPS

In [ ]:
metricsPS["condition"] = metricsPS["group"].apply(lambda x: "11mix" if "11" in str(x) else "10mix")


In [ ]:
import seaborn as sns

In [ ]:
metricsPS.to_csv("metricsPS.csv")

In [ ]:
metricsPS

In [ ]:
g=sns.boxplot(data=metricsPS, x="x",y="Richness", hue= "condition")

sns.stripplot(data=metricsPS, x="x", y="Richness", hue="condition", dodge=True, 
              jitter=True, alpha=0.6, marker="o", edgecolor="gray", linewidth=0.5)

g.legend(loc='upper right', bbox_to_anchor=(1.26, 0.9))
g.set_ylabel("Clonal expansion")
#annotator = Annotator(data=metricsPS, ax=g, pairs=[(("CRC","10mix"),("CRC","11mix")) ],  x="x", y="Richness", hue="MSI")
#annotator.configure(test='t-test_ind', text_format='full', loc='inside')
#annotator.apply_and_annotate()
#add_stat_annotation(g, data=metricsPerPatient, x="x", y="Richness", hue="MSI", width=0.8,
#                    box_pairs=[(("CRC","11mix"),("CRC","10mix")) ],
#                    test='t-test_ind', text_format='star', loc='inside', verbose=0, comparisons_correction='bonferroni')
g.grid(False)
#g.figure.savefig(resultsDir+"/TCR_cd8_perPatient_richness.png", bbox_inches="tight")
#g.figure.savefig(resultsDir+"/TCR_cd8_perPatient_richness.svg", bbox_inches="tight")

In [ ]:
## number of cells per clonotype

In [ ]:
tcr_11mix
tcr_10mix

#### Richness per cell annotation

In [ ]:
adata_final_annotation = sc.read_h5ad(
    "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/13_january_2025/mapped_data/mapped_til_colon.h5ad"
)

In [ ]:
adata_final_annotation.obs

In [ ]:
test.obs["cell_type"] = adata_final_annotation.obs["cell_type"].reindex(test.obs.index)

In [ ]:
tcr_10mix.obs["cell_type"] =test.obs["cell_type"] 

In [ ]:
tcr_11mix.obs["cell_type"] = test.obs["cell_type"] 

In [ ]:
import numpy as np

# Concatenate 'cell_type' and 'ifng' as strings for tcr_10mix
tcr_10mix.obs["Cell type (IFNG)"] = tcr_10mix.obs["cell_type"].astype(str) + tcr_10mix.obs["ifng"].astype(str)

# Concatenate 'cell_type' and 'ifng' as strings for tcr_11mix
tcr_11mix.obs["Cell type (IFNG)"] = tcr_11mix.obs["cell_type"].astype(str) + tcr_11mix.obs["ifng"].astype(str)


In [ ]:
mPerCell10mix=ir.tl.alpha_diversity(tcr_10mix, groupby="Cell type (IFNG)", target_col="clone_id", metric="normalized_shannon_entropy", inplace=False)
mPerCell11mix=ir.tl.alpha_diversity(tcr_11mix, groupby="Cell type (IFNG)", target_col="clone_id", metric="normalized_shannon_entropy", inplace=False)

In [ ]:
mPerCell10mix.sort_values(by=0, ascending=False, inplace=True)
mPerCell11mix.sort_values(by=0, ascending=False, inplace=True)

In [ ]:
mPerCell10mix

In [ ]:
mdata["airr"].obs["group"].isin(["11mix","10mix"]).copy()

In [ ]:
tcr_10mix.obs["clone_id_dedup"]=tcr_10mix.obs.clone_id.duplicated()
tcr_11mix.obs["clone_id_dedup"]=tcr_11mix.obs.clone_id.duplicated()

In [ ]:
test=pd.concat([tcr_10mix.obs,tcr_11mix.obs])
test=test[["Ambiguous" not in x for x in test.chain_pairing]]

In [ ]:
test.ifng.value_counts()

In [ ]:
test[["clone_id","clone_id_size", "clone_id_dedup","group"]]

In [ ]:
test.clone_id_size.value_counts()

In [ ]:
mu.pp.filter_obs(mdata, "airr:group", lambda x: ~np.isin(x, ["GF", "GF-plus"]))

In [ ]:
mu.pp.filter_obs(mdata, "airr:origin", lambda x: np.isin(x, ["tumor"]))

In [ ]:
mdata.update()

In [ ]:
mdata["airr"].obs

In [ ]:
mdata["airr"].obs.group.unique()

In [ ]:
import pandas as pd

# Keep only '10mix' and '11mix' and remove other categories
mdata["airr"].obs["group"] = mdata["airr"].obs["group"].astype(str)  # Ensure it's a string
mdata["airr"].obs["group"] = pd.Categorical(
    mdata["airr"].obs["group"], 
    categories=['10mix', '11mix'],  # Define only the categories to keep
    ordered=True
)

# Verify the categories after filtering
print(mdata["airr"].obs["group"].unique())  # Should only show ['10mix', '11mix']


In [ ]:
g=sns.barplot(data=mdata["airr"].obs, y="clone_id_size",  x="group", orient="v", palette=["tab:orange","tab:blue"], width=0.4)
g.legend(loc='upper right', bbox_to_anchor=(0.5, 0.9))
g.set_ylabel("Clonal expansion")


g.grid(False)

#g.figure.savefig(resultsDir+"/TCR_cd8_ifng_posVneg_msiVmss.png", bbox_inches="tight")
#g.figure.savefig(resultsDir+"/TCR_cd8_ifng_posVneg_msiVmss.svg", bbox_inches="tight")

In [ ]:
mdata["airr"].obs["ifng"] =  mdata["gex"].obs["ifng"]

In [ ]:
tcr_copy

In [ ]:
mdata["airr"].obs["cell_type"] = mdata["gex"].obs["cell_type"]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define figure size (adjust as needed)
plt.figure(figsize=(3, 3))  # Width = 8 inches, Height = 2 inches

# Define the custom color palette in the specified order
custom_palette = ["red", "blue", "orange", "lightblue", "green"]

# Create the bar plot with the custom color palette
g = sns.barplot(
    data=mdata["airr"].obs, 
    y="clone_id_size",  
    x="cell_type", 
    orient="v", 
    palette=custom_palette,  # Apply custom colors
    width=0.7
)

# Rotate x-axis labels for better readability
plt.xticks(size=10,rotation=45, ha='right')  # Rotate labels 45 degrees and align right

# Adjust legend position (if applicable)
if g.legend_:
    g.legend(loc='upper right', bbox_to_anchor=(0.5, 0.9))

# Set y-axis label
g.set_ylabel("Clonal Expansion")

# Remove grid lines
g.grid(False)

# Show the plot
plt.show()


In [ ]:
tcr_copy

In [ ]:
ir.tl.chain_qc(tcr_copy)

_ = ir.pl.group_abundance(
   tcr_copy, groupby="receptor_type", target_col="sample_id"
)


In [ ]:
ir.tl.chain_qc(mdata)

_ = ir.pl.group_abundance(
   mdata, groupby="airr:receptor_type", target_col="gex:cell_type"
)


In [ ]:
ir.tl.chain_qc(mdata)

_ = ir.pl.group_abundance(
   mdata, groupby="airr:receptor_type", target_col="gex:condition"
)


In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:condition")

In [ ]:
print(
    "Fraction of cells with more than one pair of TCRs: {:.2f}".format(
        np.sum(mdata.obs["airr:chain_pairing"].isin(["extra VJ", "extra VDJ", "two full chains", "multichain"]))
        / mdata["airr"].n_obs
    )
)

In [ ]:
mu.pl.embedding(mdata, basis="gex:umap", color="airr:chain_pairing", groups="multichain")

In [ ]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: x != "multichain")

In [ ]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: ~np.isin(x, ["orphan VDJ", "orphan VJ"]))

In [ ]:
mu.pp.filter_obs(mdata, "gex:condition", lambda x: ~np.isin(x, ["GF", "GF-plus"]))

In [ ]:
mdata

In [ ]:
ax = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:origin")

In [ ]:
mdata["gex"].obs["condition"].isin(["10mix","11mix"])

In [ ]:
ax = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:condition", normalize=True)

In [ ]:
# using default parameters, `ir_dist` will compute nucleotide sequence identity
ir.pp.ir_dist(mdata)


In [ ]:
mdata["airr"].obs["chain_pairing"]=mdata["airr"].obs["chain_pairing"].astype("string")

In [ ]:
mdata["airr"].obs.info()

In [ ]:
sc.pp.filter_genes(mdata["gex"], min_cells=1000)
sc.pp.filter_cells(mdata["gex"], min_genes=100)

In [ ]:
ir.tl.define_clonotypes(mdata, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.pp.ir_dist(
    mdata,
    metric="alignment",
    sequence="aa",
    cutoff=15,
)

In [ ]:
ir.tl.define_clonotype_clusters(mdata, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any")

In [ ]:
ir.tl.clonotype_network(mdata, min_cells=2, sequence="aa", metric="alignment")

In [ ]:
_ = ir.pl.clonotype_network(
    mdata,
    color="gex:sample_id",
    label_fontsize=9,
    panel_size=(7, 7),
    base_size=2
)


In [ ]:
import scirpy as ir
import pandas as pd

# Define the desired category order
desired_order = ['CD8_NaiveLike', 'CD8_EarlyActiv', 'CD8_EffectorMemory', 'CD8_Tpex', 'CD8_Tex']

# Convert 'cell_type' to a categorical variable with the specified order
mdata["gex"].obs["cell_type"] = pd.Categorical(
    mdata["gex"].obs["cell_type"], 
    categories=desired_order, 
    ordered=True
)
mdata.update()
# Plot clonal expansion with the specified order
_ = ir.pl.clonal_expansion(
    mdata,
    target_col="clone_id",
    groupby="gex:cell_type",
    normalize=False
)


In [ ]:
_ = ir.pl.clonal_expansion(
    mdata,
    target_col="clone_id",
    groupby="gex:condition",normalize=False
)


In [ ]:
import scirpy as ir
import pandas as pd

# Define the desired category order
desired_order = ['CD8_NaiveLike', 'CD8_EarlyActiv', 'CD8_EffectorMemory', 'CD8_Tpex', 'CD8_Tex']

# Convert 'cell_type' to a categorical variable with the specified order
mdata["gex"].obs["cell_type"] = pd.Categorical(
    mdata["gex"].obs["cell_type"], 
    categories=desired_order, 
    ordered=True
)


mdata.update()


_ = ir.pl.alpha_diversity(
    mdata,
    metric="normalized_shannon_entropy",
    groupby="gex:cell_type"
)

In [ ]:
ir.tl.clonotype_modularity(mdata, target_col="airr:cc_aa_alignment")

In [ ]:
_ = ir.pl.clonotype_modularity(mdata, base_size=20)

In [ ]:
clonotypes_top_modularity = list(
	mdata.obs.set_index("airr:cc_aa_alignment")["airr:clonotype_modularity"]
	.sort_values(ascending=False)
	.index.unique()
	.values[:1]
)

In [ ]:
mu.pl.embedding(mdata,basis="gex:tsne", color=["gex:cell_type"])
mu.pl.embedding(
    mdata,
    basis="gex:tsne",
    color="airr:cc_aa_alignment",
    groups=clonotypes_top_modularity
)
